<a href="https://colab.research.google.com/github/tasn19/scan-repro/blob/main/SCAN_repro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

Import setup files from github

In [ ]:
# Import setup files from github
%%shell

git clone https://github.com/tasn19/scan-repro.git
cd setup
git checkout v0.3.0

cp utils.py ../
cp dataset.py ../
cp models.py ../
cp criterion.py ../
cp memorybank.py ../
cp train.py ../

In [ ]:
# import libraries common to all tasks

# Pretext Task

In [ ]:
# Import all pretext stuff
import torch
import torchvision
from dataset import CustomDataset
from utils import get_transform, contrastive_evaluate
from models import get_model
from criterion import SimCLR_loss
from memorybank import MemoryBank, fill_memory_bank
from train import SimCLR_train


In [ ]:
!pip install faiss-gpu # needed for mine_nearest_neighbors -> Memory Bank

In [ ]:
# Set paths to store files
cifar_path = "/content/drive/MyDrive/Colab Notebooks/SCANmaterials/Unsupervised-Classification/datasets/cifar10"
pretext_model_path = "/content/drive/MyDrive/Colab Notebooks/SCANmaterials/Unsupervised-Classification/mymodels/simclrmodel"
knn_train_path = "/content/drive/MyDrive/Colab Notebooks/SCANmaterials/Unsupervised-Classification/mymodels/knn_train.npy"
knn_test_path = "/content/drive/MyDrive/Colab Notebooks/SCANmaterials/Unsupervised-Classification/mymodels/knn_test.npy"

In [ ]:
# Get transformations & datasets 
step = "simclr"
featuresDim = 128
numClasses = 10

# Get transformations
transform = get_transform("simclr")
base_transform = get_transform("base") # FIX !!!!!
val_transform = get_transform("validate")

# author code epochs = 500, batchsize = 512, num_workers = 8
hyperparams = {"epochs": 20, "batchsize": 512, "weight decay": 0.0001, "momentum": 0.9, "lr": 0.4, 
                   "lr decay rate": 0.1, "num_workers": 8}

# Load training set
train1_set = torchvision.datasets.CIFAR10(root = cifar_path, train = True, transform = transform, download = False) # change to True
train_set = CustomDataset(train1_set, step, base_transform = base_transform)
# For initial testing, take a small subset of dataset
indices = torch.randperm(len(train_set)).tolist() # INITIAL TESTING
train_set = torch.utils.data.Subset(train_set, indices[:1000]) # INITIAL TESTING
# enable pin_memory to speed up host to device transfer
# Probably highest possible batch_size=128 & num_workers=2 with memory limits CHECK
train_loader = torch.utils.data.DataLoader(train_set, batch_size = hyperparams["batchsize"], shuffle = True, 
                                           num_workers = hyperparams["num_workers"], pin_memory = True, drop_last = True)

# Load testing set
test_set = torchvision.datasets.CIFAR10(root = cifar_path, transform = transform, download = False)
test_set = torch.utils.data.Subset(test_set, indices[:1000]) # INITIAL TESTING
test_loader = torch.utils.data.DataLoader(test_set, batch_size = hyperparams["batchsize"], shuffle = False, 
                                          num_workers = hyperparams["num_workers"], pin_memory = True, drop_last = False)


In [ ]:
# Instantiate model
model = get_model(step)
model.cuda()

# Get criterion
criterion = SimCLR_loss(hyperparams["batchsize"])
criterion.cuda()

In [ ]:
# Build memory bank
# use dataset without augmentations for knn evaluation CHECK
base_dataset = torchvision.datasets.CIFAR10(root= cifar_path, train=True, transform=val_transform, download=True)
# subset
base_dataset = torch.utils.data.Subset(base_dataset, indices[:1000]) # remove
base_loader = torch.utils.data.DataLoader(base_dataset, batch_size = hyperparams["batchsize"], shuffle = False, 
                                          num_workers = hyperparams["num_workers"], pin_memory = True)
base_memorybank = MemoryBank(len(base_dataset), featuresDim, numClasses, temperature = 0.1)
base_memorybank.cuda()
test_memorybank = MemoryBank(len(test_set), featuresDim, numClasses, temperature = 0.1) 
test_memorybank.cuda()


In [ ]:
lr = 0.4
decay_rate = 0.1
epochs = hyperparams["epochs"]
# Instantiate SGD?? optimizer # original paper used LARS...
#params = [p for p in model.parameters() if p.requires_grad] # CHECK THIS
params = model.parameters()
optimizer = torch.optim.SGD(params, lr, momentum=0.9, weight_decay=0.0001, nesterov=False)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs, eta_min=(lr*(decay_rate**3)))

# Checkpoint - use paper code

# Train model
# add warm-up? (to reduce effect of early training)
for epoch in range(epochs):
  print('Epoch ', epoch)
  # Update scheduler (it resets every epoch)
  scheduler.step()
  lr = scheduler.get_last_lr()[0]
  #scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs, eta_min=(lr*(decay_rate**3)))
  
  print('Learning Rate ', lr)

  # Train
  SimCLR_train(train_loader, model, epoch, criterion, optimizer)
  print("SimCLR train complete")

  # Fill memory bank for knn step  
  fill_memory_bank(base_loader, model, base_memorybank)

  # Check progress
  print("Evaluating")
  top1 = contrastive_evaluate(test_loader, model, base_memorybank)
  print('Result of kNN evaluation is %.2f' %(top1)) 

  # checkpoint 

# Save model
#torch.save(model.state_dict(), my_model_path)


Mine the top 20 nearest neighbors to pass on to SCAN step

In [ ]:
k = 20
fill_memory_bank(base_loader, model, base_memorybank)
train_indices, accuracy = base_memorybank.mine_nearest_neighbors(k)
print('Accuracy of top-%d nearest neighbors on train set is %.2f' %(k, 100*accuracy))
# save positions of nearest neighbors
np.save(knn_train_path, indices)   

Mine the top 5 neighbors for validation

In [ ]:
fill_memory_bank(test_loader, model, test_memorybank)
k = 5
test_indices, accuracy = test_memorybank.mine_nearest_neighbors(k)
print('Accuracy of top-%d nearest neighbors on test set is %.2f' %(k, 100*acc))
# save positions of nearest neighbors
np.save(knn_test_path, indices)  

# SCAN

In [ ]:
# Import all SCAN stuff

In [ ]:
# Set paths to store files
SCAN_model_path = "/content/drive/MyDrive/Colab Notebooks/SCANmaterials/Unsupervised-Classification/mymodels/scanmodel"
cifar_path = "/content/drive/MyDrive/Colab Notebooks/SCANmaterials/Unsupervised-Classification/datasets/cifar10"
knn_train_path = "/content/drive/MyDrive/Colab Notebooks/SCANmaterials/Unsupervised-Classification/mymodels/knn_train.npy"

In [ ]:
step2 = "scan"
scan_hyperparams = {"epochs": 20, "batchsize": 128, "lr": 0.0001, "weight decay": 0.0001, "num_workers": 8}

# Get transformations
scan_transforms = get_transform(step2)
base_transform = get_transform("base") # FIX !!!!!
# val_transforms is the same as simclr 
val_transform = get_transform("validate")

# Load training set
train1_set_scan = torchvision.datasets.CIFAR10(root = cifar_path, train = True, transform = scan_transforms, download = False) # change to True
knn_indices = np.load(knn_train_path)
knn_indices = np.reshape(knn_indices, (-1,20)) # CHECK SHAPE
train_set_scan = NNDataset(train1_set_scan, knn_indices, numNeighbors=20, step=step2, base_transform = base_transform)
indices1 = torch.randperm(len(train_set_scan)).tolist()  # INITIAL TESTING
train_set_scan = torch.utils.data.Subset(train_set_scan, indices1[:1000]) # INITIAL TESTING
train_loader_scan = torch.utils.data.DataLoader(train_set_scan, batch_size = scan_hyperparams["batchsize"], shuffle = True, 
                                           num_workers = scan_hyperparams["num_workers"], pin_memory = True, drop_last = True)

# Load testing set
test1_set_scan = torchvision.datasets.CIFAR10(root = cifar_path, train = False, transform = val_transform, download = False)
knn_test_indices = np.load(knn_test_path) 
knn_test_indices = np.reshape(knn_test_indices, (-1,20))
test_set_scan = NNDataset(test1_set_scan, knn_test_indices, numNeighbors=5, step=step2, base_transform = base_transform)
test_set_scan = torch.utils.data.Subset(test_set_scan, indices1[:1000]) # INITIAL TESTING
test_loader_scan = torch.utils.data.DataLoader(test_set_scan, batch_size = scan_hyperparams["batchsize"], shuffle = False, 
                                          num_workers = scan_hyperparams["num_workers"], pin_memory = True, drop_last = False)

In [ ]:
# Set device to cuda if cuda is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Instantiate model
model = get_model(step2, pretrained_weights=None, numClasses=10) # add param pretrained_path?
model.to(device)

# Get criterion
criterion = SCAN_loss(entropy_weight = 5)
criterion.to(device)